# Assignment 3 - MCMC Methods in Julia
## Pranshu Gaur - 200703

### Question 1


For estimating mean of a standard Cauchy distribution using importance sampling with a normal proposal distribution, 
$\large{h(x) = x, \pi(x) = \frac{1}{\pi(x^2+1)}, g(x) = \frac{e^\frac{-x^2}{2}}{\sqrt{2\pi}}}$

$$\ \hat{\theta}_g=\ \frac{1}{N}\ \sum_{t=1}^{N} \frac{h(Z_t) \pi(Z_t)}{g(Z_t)}$$

$Var(\hat{\theta}_g)$ will be finite if ${Var(\ \frac{h(Z_t)\ \pi(Z_t)}{g(Z_t)})}$ is finite. This is finite when its second moment is finite. So, the second moment of $ \frac{h(Z_t)\ \pi(Z_t)}{g(Z_t)}$ is

$$E({\frac{h^2(Z_t)\ \pi^2(Z_t)}{g^2(Z_t)}}) = \int_{-\infty}^{+\infty} \frac{x^2 \frac{1}{\pi^2(x^2+1)^2}}{\frac{e^{-x^2}}{{2\pi}}}\frac{e^{\frac{-x^2}{2}}}{\sqrt{2\pi}}\,dx$$


$$= \int_{-\infty}^{+\infty} \frac{\sqrt{2}}{\pi\sqrt{\pi}}x^2 \frac{e^{\frac{x^2}{2}}}{(x^2+1)^2} \,dx$$

$$\geq \int_{-\infty}^{+\infty} \frac{\sqrt{2}}{\pi\sqrt{\pi}}x^2 \frac{e^{\frac{x^2}{2}}}{(x^2+2)^2} \,dx$$
                 
$$= \int_{-\infty}^{+\infty} \frac{\sqrt{2}}{\pi\sqrt{\pi}}x^2 \frac{e^{\frac{x^2}{2}}}{4(\frac{x^2}{2}+1)^2} \,dx$$

As $e^{\frac{x^2}{2}}\geq 1 + \frac{x^2}{2}$ $$\geq \int_{-\infty}^{+\infty} \frac{\sqrt{2}}{4\pi\sqrt{\pi}} \frac{x^2}{(\frac{x^2}{2}+1)} \,dx$$


$$= \int_{-\infty}^{+\infty} \frac{\sqrt{2}}{2\pi\sqrt{\pi}}\frac{x^2}{x^2+2} \,dx = \int_{-\infty}^{+\infty} \frac{\sqrt{2}}{2\pi\sqrt{\pi}}(1 - \frac{2}{x^2+2}) \,dx \to \infty $$

Since the second moment is not finite, estimator does not have finite variance.

---

### Question 2

(a) If $\large{sup \frac{f(x)}{g(x)} < \infty}$, then $\large{sup \frac{\tilde{f(x)} }{\tilde{g(x)} } = \frac{a}{b} sup \frac{f(x)}{g(x)} < \infty}$ iff $\large{\frac{a}{b}}$ is finite 

(b) To estimate the mean of target density, importance sampling has a benefit over accept-reject that it is faster because a particular proposal might get rejected in accept-reject but in importance sampling proposals are always accepted.

---

### Question 3

In [1]:
using Distributions
using Random
Random.seed!(1)
function samp_y(v,n)
    sample = []
    for i in 1:n
        x = rand(TDist(v))
        push!(sample,x)
    end
    return sample
end


samp_y (generic function with 1 method)

In [2]:
function target(v,n,x)
    val = 1
    y = samp_y(v,n)
    num = 0
    denom = 0
    for i in 1:n
        val *=(1 + ((y[i] - x)*(y[i] - x)/v))^(-(v+1)/2)
    end
    return val
end

function first_moment(v,n,N)
    num = 0
    denom = 0
    
    for i in 1:N
        x = rand(Normal(0,1))
        fac = (sqrt(2*pi))
        sam = fac*target(v,n,x)
        num += x*sam
        denom += sam
    end
    return num/denom
end

function second_moment(v,n,N)
    num = 0
    denom = 0
    
    for i in 1:N
        x = rand(Normal(0,1))
        fac = (sqrt(2*pi))
        sam = fac*target(v,n,x)
        num += x*x*sam
        denom += sam
    end
    return num/denom
end

e1 = first_moment(5,50,10000)
e2 = second_moment(5,50,10000)
var = e2-(e1)^2
println("For v = 5: Expectation = ", e1,";   Variance = ",var)


For v = 5: Expectation = -0.13206374634478935;   Variance = 0.022988940648935332


In [3]:
e1 = first_moment(1,50,10000)
e2 = second_moment(1,50,10000)
var = e2-(e1)^2
println("For v = 1: Expectation = ", e1,";   Variance = ",var)

For v = 1: Expectation = -0.2956177611094221;   Variance = -0.06938755390440102


In [4]:
e1 = first_moment(2,50,10000)
e2 = second_moment(2,50,10000)
var = e2-(e1)^2
println("For v = 2: Expectation = ", e1,";   Variance = ",var)

For v = 2: Expectation = 0.22627280489282034;   Variance = 0.0040950556326433685


#### In all the cases, the variance is finite. 

---

### Question  4

We take the pdf of Gamma$(\alpha,\beta)$ as $\large{\frac{\beta^\alpha}{\Gamma(\alpha)}x^{\alpha-1}  e^{-\beta x}}$ 

So, $$f(\lambda) = \frac{\beta^\alpha}{\Gamma(\alpha)}\lambda^{\alpha-1}  e^{-\beta \lambda}$$

Now, $$f(Y_1,Y_2,...,Y_n|\lambda) = \prod_{i=1}^{n} e^{-\lambda} \frac{\lambda^{Y_i}}{{Y_i}!}$$

Therefore, $$f(Y_1,Y_2,...,Y_n|\lambda) \propto e^{-n\lambda} \lambda^{\sum_{n=1}^{\infty} Y_i}$$


We need to find $$f(\lambda|Y_1,Y_2,...,Y_n)$$

$$\propto f(Y_1,Y_2,...,Y_n|\lambda) \times f(\lambda) $$

$$\propto e^{-n\lambda} \lambda^{\sum_{n=1}^{\infty} Y_i} \frac{\beta^\alpha}{\Gamma(\alpha)}\lambda^{\alpha-1}  e^{-\beta \lambda}$$

Let $Y = \sum_{n=1}^{\infty} Y_i$
Then, $$f(\lambda|Y_1,Y_2,...,Y_n) \propto \lambda^{(Y+\alpha)-1} e^{-(\beta + n)\lambda}$$

Hence, the posterior distribution of $\lambda$ must be Gamma$(Y+\alpha,\beta+n)$